In [1]:
## Load context
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
import re
import os
hostname = os.uname()[1]

from pyspark.sql import SparkSession

# BEN'S MASTER: 192.168.2.87
# OUR MASTER:   192.168.2.203

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.203:7077") \
        .appName(f"main; hostname: {hostname}")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext
spark_context.uiWebUrl

'http://host-192-168-2-247-ldsa:4040'

In [ ]:
#PySpark API for fetching a RDD

#TODO: Placeholder params. Set this up when we have the cluster running
rdd = spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.87:9000/europarl/europarl-v7.sv-en.en',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text')\
.cache() # Keep this RDD in memory!

In [1]:
#PySQL API for fetching a DataFrame

#TODO: Placeholder params. Set this up when we have the cluster running
df = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.2.87:9000/parking-citations.csv')\
    .cache()

df.show()

NameError: name 'spark_session' is not defined